SUBMISSION -- ANANYA MUKHERJEE

In [12]:
import pandas as pd
df_cust = pd.read_csv('.\\archive\\olist_customers_dataset.csv')
#print(df_cust.head())
df_order = pd.read_csv('.\\archive\\olist_orders_dataset.csv')
#print(df_order.head())
df_pay = pd.read_csv('.\\archive\\olist_order_payments_dataset.csv')
#print(df_pay.head())

#keeping only select required columns
df_cust = df_cust[['customer_id','customer_unique_id','customer_city','customer_state']]
df_order = df_order[['order_id','customer_id','order_status','order_purchase_timestamp']]
df_pay = df_pay[['order_id','payment_value']]
print(df_cust.describe())
print(df_order.describe())
print(df_pay.describe())

# doing join of df_order and df_customers on customer_id
df_merged = pd.merge(df_order, df_cust, on='customer_id', how='inner')
# doing join of df_merged and df_pay on order_id
df_merged = pd.merge(df_merged, df_pay, on='order_id', how='inner')
print(df_merged.head())



                             customer_id                customer_unique_id  \
count                              99441                             99441   
unique                             99441                             96096   
top     06b8999e2fba1a1fbc88172c00ba8bc7  8d50f5eadf50201ccdcedfb9e2ac8455   
freq                                   1                                17   

       customer_city customer_state  
count          99441          99441  
unique          4119             27  
top        sao paulo             SP  
freq           15540          41746  
                                order_id                       customer_id  \
count                              99441                             99441   
unique                             99441                             99441   
top     e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
freq                                   1                                 1   

       order_status order_pu

<h3>Notes on Data</h3>

There are 99441 instances of customer ids, which correspond to 96096 unique customer ids. These customers belong to 4119 cities across 27 states. 

There are 99441 unique order ids, with the status belonging to 8 categories

There are 1038886 payments with Mean payment value is 154.1

In [2]:
#keeping only orders with status = 'delivered'
df_merged = df_merged[df_merged['order_status']=='delivered']

#convert order_purchase_timestamp to datetime
df_merged['order_purchase_timestamp'] = pd.to_datetime(df_merged['order_purchase_timestamp'])

#create reference date as max order_purchase_timestamp
reference_date = df_merged['order_purchase_timestamp'].max()
print(reference_date)

2018-08-29 15:00:37


<h3>Handling Edge cases: </h3>
Multiple payments for 1 order id - since the feature of Frequency is based by counting unique order ids, the payments will be aggregated and therefore the aggregation of payments to an order_id does not need to be handled separately
Orders with non-delivered status have been excluded above - only rows of df_merged where order_status=='delivered' is selected
Customers with no delivered orders will also have been eliminated from the rows of df_merged with the above selection

<b>On unique customer ids :</b> There are 96096 unique_customer_id, for 99441 customer ids. Since we are analyzing the behaviour of a customer (and not an id), we should use the unique identifier to aggregate all his/her transactions. Hence we use this for creating features in subsequent steps.

<h4>Feature engineering:</h4>

Recency, Frequency and Monetary Value are created and stored in a new dataframe df_rfm
Finally the target label 'churn' is created - which in our case is directly dependent on the value of one independent variable 'recency'


In [3]:
#develop features of recency, frequency, monetary value for each customer_unique_id
#group by customer_unique_id
df_rfm = df_merged.groupby('customer_unique_id').agg({
    'order_purchase_timestamp': lambda x: (reference_date - x.max()).days,
    'order_id': 'nunique',
    'payment_value': 'sum'
}).reset_index()
df_rfm.columns = ['customer_unique_id', 'recency', 'frequency', 'monetary_value']

#add a target label churn if recency > 180 days
df_rfm['churn'] = df_rfm['recency'].apply(lambda x: 1 if x > 180 else 0)

print(df_rfm.head())


                 customer_unique_id  recency  frequency  monetary_value  churn
0  0000366f3b9a7992bf8c76cfdf3221e2      111          1          141.90      0
1  0000b849f77a49e4a4ce2b2a4ca5be3f      114          1           27.19      0
2  0000f46a3911fa3c0805444483337064      536          1           86.22      1
3  0000f6ccb0745a6a4b88665a16c9f078      320          1           43.62      1
4  0004aac84e0df4da2b147fca70cf8255      287          1          196.89      1


. 
<h3>Logistic Regression</h3>

Classifier model has been trained with 70% of the data - 30%  kept aside for test. (random seed = 42 used to generate the train vs test data sets). 
All 3 features have been used to predict churn in this model. It has given a high accuracy of nearly 100%.

However, should we have used 'recency' as a independent variable? Such a model will clearly show high accuracy as recency fully predicts churn. 
A more realistic model would be to try build a classifier of 'churn', based only on 'frequency' and 'monetary value'. 
This has been implemented below too, and (obviously) shows a lower accuracy

In [7]:
# develop a logistic regression model to predict churn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

#split the data into train and test
X = df_rfm[['recency','frequency', 'monetary_value']]  #features including recency
y = df_rfm['churn']  #target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
#train the model
model = LogisticRegression()
model.fit(X_train, y_train)
#predict on test data
y_pred = model.predict(X_test)

# report accuracy, precision, recall, f1-score
accuracy = (y_test == y_pred).mean()
print(f'Accuracy: {accuracy}')
precision = (y_test[y_pred == 1] == 1).mean()
print(f'Precision: {precision}')
recall = (y_pred[y_test == 1] == 1).mean()
print(f'Recall: {recall}')
f1_score = 2 * (precision * recall) / (precision + recall)
print(f'F1-score: {f1_score}')
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.9998928877463582
Precision: 0.9999396900066341
Recall: 0.9998793872874201
F1-score: 0.9999095377378403

Confusion Matrix:
[[11425     1]
 [    2 16580]]


<h3>Logistic Regression - part 2</h3>

Only 2 features have been used to predict churn in this model. I have deliberately left out 'recency' as that is obviously the complete predictor of churn in my training data. This model has given a much lower accuracy of around 60%


In [8]:
#split the data into train and test
X = df_rfm[['frequency', 'monetary_value']]  #features excluding recency
y = df_rfm['churn']  #target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
#train the model
model = LogisticRegression()
model.fit(X_train, y_train)
#predict on test data
y_pred = model.predict(X_test)

# report accuracy, precision, recall, f1-score
accuracy = (y_test == y_pred).mean()
print(f'Accuracy: {accuracy}')
precision = (y_test[y_pred == 1] == 1).mean()
print(f'Precision: {precision}')
recall = (y_pred[y_test == 1] == 1).mean()
print(f'Recall: {recall}')
f1_score = 2 * (precision * recall) / (precision + recall)
print(f'F1-score: {f1_score}')
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.5922593544701514
Precision: 0.5923038409269723
Recall: 0.998793872874201
F1-score: 0.7436242816091954

Confusion Matrix:
[[   26 11400]
 [   20 16562]]


<h3>Random Forest Classification</h3>

Classifier model has been trained with 70% of the data - 30%  kept aside for test. (random seed = 42 used to generate the train vs test data sets). 
All 3 features have been used to predict churn in this model. It has given a high accuracy of 100% (better than the logistic regression scores).

However, a more realistic model to build a classifier of 'churn', based only on 'frequency' and 'monetary value' has been implemented below too, and (obviously) shows a lower accuracy

In [ ]:
#develop a random forest classifier for churn
from sklearn.ensemble import RandomForestClassifier

#split the data into train and test
X = df_rfm[['recency','frequency', 'monetary_value']]  #features including recency
y = df_rfm['churn']  #target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
#train the random  forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
#predict on test data
y_pred = model.predict(X_test)
# report accuracy, precision, recall, f1-score
accuracy = (y_test == y_pred).mean()
print(f'Accuracy: {accuracy}')
precision = (y_test[y_pred == 1] == 1).mean()
print(f'Precision: {precision}')
recall = (y_pred[y_test == 1] == 1).mean()
print(f'Recall: {recall}')
f1_score = 2 * (precision * recall) / (precision + recall)
print(f'F1-score: {f1_score}')
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0

Confusion Matrix:
[[11426     0]
 [    0 16582]]


<h3>Random Forest - part 2</h3>

Only 2 features have been used to predict churn in this model. I have deliberately left out 'recency' as that is obviously the complete predictor of churn in my training data. This model has given a much lower accuracy of around 74%. This is again higher than the logistic regression model made with 2 independent features.


In [10]:
#split the data into train and test
X = df_rfm[['frequency', 'monetary_value']]  #features excluding recency
y = df_rfm['churn']  #target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
#train the random  forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
#predict on test data
y_pred = model.predict(X_test)
# report accuracy, precision, recall, f1-score
accuracy = (y_test == y_pred).mean()
print(f'Accuracy: {accuracy}')
precision = (y_test[y_pred == 1] == 1).mean()
print(f'Precision: {precision}')
recall = (y_pred[y_test == 1] == 1).mean()
print(f'Recall: {recall}')
f1_score = 2 * (precision * recall) / (precision + recall)
print(f'F1-score: {f1_score}')
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.7370751213938874
Precision: 0.7731421121251629
Recall: 0.7867567241587263
F1-score: 0.7798900047824007

Confusion Matrix:
[[ 7598  3828]
 [ 3536 13046]]


<h3>Reproducibility notes </h3>

<b>Random seed</b> used has been 42 in all cases - for splitting train/test samples and for the random forest state

<b>Python libraries</b> used are pandas version 2.3.2 , scikit-learn / sklearn version 1.7.2

<b>Python</b> version 3.13.7